In [72]:
# #Set the enviroment variables
# import os
# os.environ['KAGGLE_USERNAME'] = "dlromanoff"
# os.environ['KAGGLE_KEY'] = "a63612e0bc2ceaee594bcec8d8f498b6"

In [148]:
# Imports needed or not
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from mpl_toolkits.mplot3d import Axes3D

In [392]:
# Read the csv's
train_features = pd.read_csv('train_features.csv')
test_features = pd.read_csv('test_features.csv')
train_labels = pd.read_csv('train_labels.csv')
train_features.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [393]:
train_features.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [394]:
# Checking for values
train_labels.status_group.value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [395]:
# Merging dataframes by id to make it easier to use
train_new = pd.merge(train_labels, train_features, on='id')
print(train_new.shape)
train_new.head()

(59400, 41)


,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [396]:
train_new = train_new.drop(['funder', 'installer', 'subvillage', 'public_meeting', 'scheme_management', 'scheme_name',
                            'permit'], axis=1)
train_new.isnull().sum()

id                       0
status_group             0
amount_tsh               0
date_recorded            0
gps_height               0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
recorded_by              0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
waterpoint_type_group    0
dtype: int64

In [397]:
# Making a list of the columns in the new df
print(list(train_new))

['id', 'status_group', 'amount_tsh', 'date_recorded', 'gps_height', 'longitude', 'latitude', 'wpt_name', 'num_private', 'basin', 'region', 'region_code', 'district_code', 'lga', 'ward', 'population', 'recorded_by', 'construction_year', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']


In [398]:
test_new = test_features.drop(['funder', 'installer', 'subvillage', 'public_meeting', 'scheme_management', 'scheme_name',
                            'permit'], axis=1)
test_new.isnull().sum()

id                       0
amount_tsh               0
date_recorded            0
gps_height               0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
recorded_by              0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
waterpoint_type_group    0
dtype: int64

In [399]:
# Making all objects into numbers
def encode_cat(df):
    for col_name in df.columns:
        if(df[col_name].dtype == 'object'):
            df[col_name]= df[col_name].astype('category')
            df[col_name] = df[col_name].cat.codes
    return df

train_new = encode_cat(train_new)
train_new.head()

,id,status_group,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,0,6000.0,47,1390,34.938093,-9.856322,37399,0,1,...,0,6,2,1,1,8,6,0,1,1
1,8776,0,0.0,309,1399,34.698766,-2.147466,37195,0,4,...,2,6,2,2,2,5,3,1,1,1
2,34310,0,25.0,300,686,37.460664,-3.821329,14572,0,5,...,5,6,2,1,1,0,1,1,2,1
3,67743,2,0.0,272,263,38.486161,-11.155298,37285,0,7,...,2,6,2,0,0,3,0,0,2,1
4,19728,0,0.0,104,0,31.130847,-1.825359,35529,0,4,...,2,6,2,3,3,5,3,1,1,1


In [400]:
test_new = encode_cat(test_new)
test_new.head()

,id,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,region,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,255,1996,35.290799,-4.059696,633,0,0,8,...,2,6,2,3,3,5,3,1,6,5
1,51630,0.0,255,1569,36.656709,-3.309214,1727,0,5,0,...,2,6,2,2,2,8,6,0,1,1
2,17168,0.0,252,1567,34.767863,-5.004344,9483,0,0,18,...,2,6,2,2,2,5,3,1,6,5
3,45559,0.0,242,267,38.058046,-9.418672,5467,0,7,7,...,6,6,2,0,0,7,5,0,6,5
4,49871,500.0,306,1260,35.006123,-10.950412,5573,0,7,16,...,1,6,2,1,1,8,6,0,1,1


In [401]:
# Assigning X and y values for Logistic Regression
X = train_new.drop(['status_group'], axis=1)
y = train_new['status_group']

In [402]:
# Fitting the model
log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)
# y_pred = log.predict(test_new)

/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/danielleromanoff/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6282996632996632

In [412]:
# X['id'].value_counts()

In [414]:
# Creating the dataframe for submission
df_submission = pd.DataFrame(X['id'], y)
df_submission.head()

,id
status_group,
0,69572
0,69572
0,69572
2,34310
0,69572


In [374]:
# Making the 'status_group' a column
df_submission.reset_index(level=0, inplace=True)
df_submission.head()

,status_group,id
0,0,69572
1,0,69572
2,0,69572
3,2,34310
4,0,69572


In [375]:
# Returning zero to 'functional'
df_submission['status_group'] = df_submission['status_group'].replace(0, 'functional')
df_submission.head()

,status_group,id
0,functional,69572
1,functional,69572
2,functional,69572
3,2,34310
4,functional,69572


In [391]:
# Switching status_group and id to have in the right order
df_submission = df_submission[['id', 'status_group']]
df_submission.head(20)

,id,status_group
0,69572,functional
1,69572,functional
2,69572,functional
4,69572,functional
5,69572,functional
9,69572,functional
10,69572,functional
11,69572,functional
12,69572,functional
13,69572,functional


In [390]:
# Dropping all status_group values that are not equal to 'functional'
df_submission = df_submission[df_submission["status_group"] == 'functional']
df_submission.head()

,id,status_group
0,69572,functional
1,69572,functional
2,69572,functional
4,69572,functional
5,69572,functional
9,69572,functional
10,69572,functional
11,69572,functional
12,69572,functional
13,69572,functional


In [405]:
df_submission.to_csv(path_or_buf="/Users/danielleromanoff/Documents/Lambda/DS-Unit-2-Project-Week/Submission_1.csv", index=False)